In [1]:
!pip install gqlalchemy
!pip install nltk

  Using cached gqlalchemy-1.5.1-py3-none-any.whl.metadata (7.2 kB)
  Using cached adlfs-2023.12.0-py3-none-any.whl.metadata (7.1 kB)
  Using cached dacite-1.8.1-py3-none-any.whl.metadata (15 kB)
  Using cached neo4j-4.4.12-py3-none-any.whl
  Using cached networkx-2.8.8-py3-none-any.whl.metadata (5.1 kB)
  Using cached pydantic-2.7.1-py3-none-any.whl.metadata (107 kB)
  Using cached pymgclient-1.3.1.tar.gz (125 kB)
  Preparing metadata (setup.py) ... done
  Using cached azure_core-1.30.1-py3-none-any.whl.metadata (37 kB)
  Using cached azure_datalake_store-0.0.53-py2.py3-none-any.whl.metadata (19 kB)
  Using cached azure_identity-1.16.0-py3-none-any.whl.metadata (76 kB)
  Using cached azure_storage_blob-12.20.0-py3-none-any.whl.metadata (26 kB)
  Using cached fsspec-2024.5.0-py3-none-any.whl.metadata (11 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached pydantic_core-2.18.2-cp311-cp311-macosx_11_0_arm64.whl.metadata (6.5 kB)
  Using cached msal-1.

Failed to build pymgclient
ERROR: Could not build wheels for pymgclient, which is required to install pyproject.toml-based projects


In [3]:
!pip install gqlalchemy

from gqlalchemy import Memgraph

# Connect to Memgraph using the actual server address and port
memgraph = Memgraph("127.0.0.1", 7687)


  Using cached gqlalchemy-1.5.1-py3-none-any.whl.metadata (7.2 kB)
  Using cached adlfs-2023.12.0-py3-none-any.whl.metadata (7.1 kB)
  Using cached dacite-1.8.1-py3-none-any.whl.metadata (15 kB)
  Using cached neo4j-4.4.12-py3-none-any.whl
  Using cached networkx-2.8.8-py3-none-any.whl.metadata (5.1 kB)
  Using cached pydantic-2.7.1-py3-none-any.whl.metadata (107 kB)
  Using cached pymgclient-1.3.1.tar.gz (125 kB)
  Preparing metadata (setup.py) ... done
  Using cached azure_core-1.30.1-py3-none-any.whl.metadata (37 kB)
  Using cached azure_datalake_store-0.0.53-py2.py3-none-any.whl.metadata (19 kB)
  Using cached azure_identity-1.16.0-py3-none-any.whl.metadata (76 kB)
  Using cached azure_storage_blob-12.20.0-py3-none-any.whl.metadata (26 kB)
  Using cached fsspec-2024.5.0-py3-none-any.whl.metadata (11 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached pydantic_core-2.18.2-cp311-cp311-macosx_11_0_arm64.whl.metadata (6.5 kB)
  Using cached msal-1.

Failed to build pymgclient
ERROR: Could not build wheels for pymgclient, which is required to install pyproject.toml-based projects


ModuleNotFoundError: No module named 'gqlalchemy'

In [ ]:
#number of party members
results = memgraph.execute_and_fetch(
    """
    MATCH (p:Party)<-[:MEMBER_OF]-(u:User)
    RETURN p.ident AS PartyName, COUNT(u) AS MemberCount
    ORDER BY MemberCount DESC;
    """
)
partyMembNum = pd.DataFrame([result for result in results])
partyMembNum

In [ ]:
#get tweet language statistics per party
results = memgraph.execute_and_fetch(
    """
    MATCH (p:Party)<-[:MEMBER_OF]-(u:User)-[:POSTED]->(t:Tweet)
    WHERE p.ident IN ['N-VA', 'Ecolo-Groen', 'PS', 'VB', 'MR', 'Open Vld', 'cd&v', 'PVDA-PTB', 'Vooruit', 'Les Engagés', 'DéFI', 'ONAFH']
    RETURN p.ident AS PartyIdent, t.lang AS Language, COUNT(t) AS TweetCount
    ORDER BY PartyIdent, TweetCount DESC;
    """
)
partyTweeLang = pd.DataFrame([result for result in results])
partyTweeLang

In [ ]:
# Politician with the most comments
query = """
MATCH (u:User)-[:MEMBER_OF]->(p:Party)
MATCH (u)-[:POSTED]->(t:Tweet)
WHERE t.is_quote IS NULL OR t.is_quote = false
MATCH (t)<-[:REPLY_TO]-(reply:Tweet)
RETURN u.name AS UserName, p.ident AS PartyName, COUNT(reply) AS ReplyCount
ORDER BY ReplyCount DESC
"""

results = memgraph.execute_and_fetch(query)

import pandas as pd

comtReced = pd.DataFrame([result for result in results])
comtReced.head(20)


In [ ]:
# politician with the most likes
query = """
MATCH (u:User)-[:MEMBER_OF]->(p:Party)
MATCH (u)-[:POSTED]->(t:Tweet)
WHERE t.is_quote IS NULL OR t.is_quote = false
RETURN u.name AS UserName, p.ident AS PartyName, SUM(t.favorite_count) AS LikeCount
ORDER BY LikeCount DESC
"""

results = memgraph.execute_and_fetch(query)

likeReced = pd.DataFrame([result for result in results])
likeReced.head(20)


In [ ]:
# politician with the most retweets
query = """
MATCH (u:User)-[:MEMBER_OF]->(p:Party)
MATCH (u)-[:POSTED]->(t:Tweet)
WHERE t.is_quote IS NULL OR t.is_quote = false
RETURN u.name AS UserName, p.ident AS PartyName, SUM(t.retweet_count) AS RetweetCount
ORDER BY RetweetCount DESC
"""

results = memgraph.execute_and_fetch(query)

retweetReced = pd.DataFrame([result for result in results])
retweetReced.head(20)


In [ ]:
#analyse mutual mention count
query = """
MATCH (u1:User)-[:MEMBER_OF]->(p1:Party),
      (u2:User)-[:MEMBER_OF]->(p2:Party),
      (u1)-[:POSTED]->(t1:Tweet)-[:MENTIONS]->(u2),
      (u2)-[:POSTED]->(t2:Tweet)-[:MENTIONS]->(u1)
RETURN u1.name AS User1, p1.ident AS Party1, u2.name AS User2, p2.ident AS Party2, COUNT(*) AS MutualMentionCount
ORDER BY MutualMentionCount DESC
LIMIT 20
"""

results = memgraph.execute_and_fetch(query)

import pandas as pd

mutual_mention_relations = pd.DataFrame([result for result in results])
mutual_mention_relations.head(20)



In [ ]:
#Analyze Theo Francken MP
query = """
MATCH (u:User {name: 'Theo Francken MP'})-[:POSTED]->(t:Tweet)
WHERE t.created_at >= '2024-01-01' AND t.created_at <= '2024-05-31'
WITH t, substring(t.created_at, 0, 7) AS month
RETURN month, collect(t.full_text) AS tweets, sum(toInteger(t.retweet_count)) AS total_retweets, sum(toInteger(t.reply_count)) AS total_comments, sum(toInteger(t.favorite_count)) AS total_likes
ORDER BY month
"""

results = memgraph.execute_and_fetch(query)
# print results
for result in results:
    print(result)

In [ ]:
# Re-run the query
results = memgraph.execute_and_fetch(query)

data = []
for result in results:
    data.append(result)

df = pd.DataFrame(data)
df['month'] = pd.to_datetime(df['month'])
print(df)



In [ ]:
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from collections import Counter
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
import pandas as pd

# Download stopwords list
nltk.download('stopwords')
nltk.download('punkt')

# Define stopwords
stop_words = set(stopwords.words('english'))  # Add English stopwords
stop_words.update(stopwords.words('dutch'))   # Add Dutch stopwords
stop_words.update(stopwords.words('french'))  # Add French stopwords

# Add more Dutch stopwords
dutch_stopwords = ["de", "en", "van", "ik", "te", "dat", "die", "in", "een", "hij", "het", "niet", "zijn", "is", "was", "op",
                   "aan", "met", "als", "voor", "had", "er", "maar", "om", "hem", "dan", "zou", "of", "wat", "mijn", "men",
                   "dit", "zo", "door", "over", "ze", "zich", "bij", "ook", "tot", "je", "mij", "uit", "der", "daar", "haar",
                   "naar", "heb", "hoe", "heeft", "hebben", "deze", "u", "want", "nog", "zal", "me", "zij", "nu", "ge", "geen",
                   "omdat", "iets", "worden", "toch", "al", "waren", "veel", "meer", "doen", "toen", "moet", "ben", "zonder",
                   "kan", "hun", "dus", "alles", "onder", "ja", "eens", "hier", "wie", "werd", "altijd", "doch", "wordt", "wezen",
                   "kunnen", "ons", "zelf", "tegen", "na", "reeds", "wil", "kon", "niets", "uw", "iemand", "geweest", "andere","onze","alle","elke",
                   "jaren", "wel", "den", "dag", "jaar", "velen", "neen","helemaal","jullie","vandaag","heel","hele","doet","bent","waar","wij"]

# Add more French stopwords
french_stopwords = ["au", "aux", "avec", "ce", "ces", "dans", "de", "des", "du", "elle", "en", "et", "eux", "il", "je", "la", "le",
                    "leur", "lui", "ma", "mais", "me", "même", "mes", "moi", "mon", "ne", "nos", "notre", "nous", "on", "ou", "par",
                    "pas", "pour", "qu", "que", "qui", "sa", "se", "ses", "son", "sur", "ta", "te", "tes", "toi", "ton", "tu", "un",
                    "une", "vos", "votre", "vous", "c", "d", "j", "l", "à", "m", "n", "s", "t", "y", "été", "étée", "étées", "étés",
                    "étant", "suis", "es", "est", "sommes", "êtes", "sont", "serai", "seras", "sera", "serons", "serez", "seront",
                    "serais", "serait", "serions", "seriez", "seraient", "étais", "était", "étions", "étiez", "étaient", "fus",
                    "fut", "fûmes", "fûtes", "furent", "sois", "soit", "soyons", "soyez", "soient", "fusse", "fusses", "fût",
                    "fussions", "fussiez", "fussent"]

# Update stopwords
stop_words.update(dutch_stopwords)
stop_words.update(french_stopwords)

additional_stopwords = {"rt", "co"}  # Add additional stopwords
stop_words.update(additional_stopwords)

# Define the cleaning function
def clean_text(text):
    text = re.sub(r'http\S+', '', text)  # Remove URLs
    text = re.sub(r'\d+', '', text)  # Remove digits
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    text = text.lower()  # Convert to lowercase
    words = word_tokenize(text)  # Tokenize
    words = [word for word in words if word not in stop_words]  # Remove stopwords
    return words

# Clean tweet text and convert to lowercase
df['cleaned_tweets'] = df['tweets'].apply(lambda tweets: [clean_text(tweet.lower()) for tweet in tweets])
df['all_cleaned_words'] = df['cleaned_tweets'].apply(lambda tweets: [word for tweet in tweets for word in tweet])
df['top_keywords'] = df['all_cleaned_words'].apply(lambda words: [item[0] for item in Counter(words).most_common(10)])

# Visualize retweets, comments, and likes count
plt.figure(figsize=(14, 8))
plt.plot(df['month'], df['total_retweets'], marker='o', label='Retweets')
plt.plot(df['month'], df['total_comments'], marker='o', label='Comments')
plt.plot(df['month'], df['total_likes'], marker='o', label='Likes')
plt.title('Monthly Engagement Metrics for Theo Francken MP')
plt.xlabel('Month')
plt.ylabel('Count')
plt.legend()
plt.grid(True)
plt.xticks(df['month'], rotation=45)
plt.tight_layout()
plt.show()

# Visualize word cloud for each month
for index, row in df.iterrows():
    text = ' '.join(row['all_cleaned_words'])
    wordcloud = WordCloud(max_font_size=50, max_words=100, background_color="white").generate(text)

    plt.figure(figsize=(8, 8))
    plt.imshow(wordcloud, interpolation="bilinear")
    plt.axis("off")
    plt.title(f'Word Cloud for {row["month"].strftime("%B %Y")}')
    plt.show()
